In [22]:
%reload_ext autoreload
%autoreload 2

import os
import sys

import numpy as np
import astropy.units as u
import astropy.constants as const
from astropy.cosmology import Planck18 as cosmo

sys.path.append(os.environ['DH_DIR'])
from darkhistory.history.reionization import alphaA_recomb
from darkhistory import physics as phys

sys.path.append(os.environ['DM21CM_DIR'])
from dm21cm.evolve import get_z_edges

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import colormaps as cms
mpl.rc_file("../matplotlibrc")

/n/home07/yitians/.conda/envs/dm21cm/lib/python3.11/site-packages/py21cmfast/_cfg.py:58: UserWarning: Your configuration file is out of date. Updating...
  warnings.warn(
/n/home07/yitians/.conda/envs/dm21cm/lib/python3.11/site-packages/py21cmfast/_cfg.py:42: UserWarning: Your configuration file is out of date. Updating...
  warnings.warn("Your configuration file is out of date. Updating...")


### constants

In [87]:
YHe = 0.245
chi = (YHe/4) / (1 - YHe)
f_H = 1 / (1 + chi) # agrees with 21cmFAST 1e-6

n_B0 = (cosmo.critical_density0 * cosmo.Ob0 / const.m_p).to('1/cm^3').value
n_A0 = 2.0553406318169421e-07

### terms

In [88]:
def dt_dz(z):
    """[s]"""
    return 1 / ((1+z) * cosmo.H(z).to('1/s').value)

def dxe_dt_recomb(xe_21, z, Tk):
    """[1/s]"""
    # clumping_factor = 1
    # peebles_C = 1
    # alpha is in [cm^3 / s]
    n_A = n_A0 * (1 + z)**3
    return alphaA_recomb('HII', Tk * phys.kB) * xe_21**2 * n_A * f_H

### values

In [89]:
# before Tk=909.59644 xe=0.023477616
# input_heating 0.0
# This is commented out: dspec_dzp = - dxe_dzp * T / (1+x_e); need to put it back in
# box_ct = 0
# dzp = -4.522629e-01
# zp = 4.422620e+01
# dt_dzp = -5.958076e+13
# dadia_dzp * dzp = -1.819196e+01
# dspec_dzp * dzp = 0.000000e+00
# dcomp_dzp * dzp = -5.101898e+01
# dxheat_dzp * dzp = 0.000000e+00
# input heating = 0.000000e+00
# dxion_source = 0.000000e+00
# dxion_sink = 5.577483e-04
# T = 9.095964e+02
# alpha_A(T) = 2.135282e-12
# global_params.CLUMPING_FACTOR = 1.000000e+00
# x_e = 2.347762e-02
# f_H = 9.249617e-01
# prefactor_1 = 1.901315e-02
# curr_delNL0 = -2.117730e-06
# growth_factor_zp = 2.814553e-02
# input ionization = 0.000000e+00

In [90]:
# test values
xe_21 = xHII = xHeII = 0.023477616
xe_DH = xe_21 * (1 + chi)
print('xe_DH', xe_DH)
Tk = 909.59644 # [K]
z_edges = get_z_edges(45, 5, 1.01)
z_prev = z_edges[1]
z = z_edges[2]
z_next = z_edges[3]
print('z', z)

xe_DH 0.02538225703311258
z 44.22620305254949


In [91]:
dxe_dt_recomb(xe_21, z, Tk)

2.0698626023882593e-17

In [82]:
5.577483e-04 / -4.522629e-01 /  -5.958076e+13

2.0698613415150128e-17

In [53]:
z_edges[1] - z_edges[2]

0.4522620305254961

In [54]:
z_edges[2]

44.22620305254949

In [51]:
# dz
print(z_prev - z)
print(z - z_next)

0.4522620305254961
0.44778418863910474


In [42]:
dzp = -3.170662e-01
zp = 4.467847e+01

In [45]:
np.abs(cosmo.age(45) - cosmo.age(z)).to('s').value / (45-z)

57133033263582.65

In [60]:
print(f'dt_dz = {dt_dz(z_edges[2]):.8e} s')

dt_dz = 5.90925788e+13 s


In [40]:
print(f'dxe_dt_recomb = {dxe_dt_recomb(xe_21, z, Tk):.8e} s^-1')

dxe_dt_21 = 8.04238909e-22 s^-1


In [61]:
cosmo.Om0

0.30966

In [65]:
import py21cmfast as p21c

In [70]:
c = p21c.CosmoParams()

In [69]:
a

CosmoParams(OMb:0.04897, OMm:0.3096, POWER_INDEX:0.9665, SIGMA_8:0.8102, hlittle:0.6766)

In [73]:
p21c.global_params.OMn, p21c.global_params.OMr

(0.0, 8.600000001024455e-05)

In [74]:
def dt_dz_21(z):
    Ho = 2.19265762e-18
    Ode0 = 1 - c.OMm - p21c.global_params.OMr
    x = np.sqrt( Ode0/c.OMm ) * (1+z)**(-3.0/2.0)
    dxdz = np.sqrt( Ode0/c.OMm ) * (1+z)**(-5.0/2.0) * (-3.0/2.0)
    const1 = 2 * np.sqrt( 1 + c.OMm/Ode0 ) / (3.0 * Ho)

    numer = dxdz * (1 + x*(x**2 + 1)**(-0.5))
    denom = x + np.sqrt(x**2 + 1)
    return (const1 * numer / denom)

In [76]:
z_edges[2]

44.22620305254949

In [75]:
dt_dz_21(4.422620e+01)

-59579997599628.87

In [ ]:
-5.958076e+13
-5.9582559615339.62